# Demonstration of the fics implementation of finite inverse categories

## Finite Inverse Categories

In [ ]:
import networkx as nx
from fics import TypedFunction
from fics import FiniteInverseCategory, FICStructure, FICStructureHomomorphism

Let us encode the simple example of $\mathcal{L}_{\text{rg}}$ to get us started.

$\mathcal{L}_{\text{rg}}$ is the finite inverse category with:
* objects $O,A,I$
* arrows $d,c : A \rightarrow O$, $i: I \rightarrow A$
* relations $id = ic$ (in diagrammatic order of compositions)

In [ ]:
G = nx.MultiDiGraph()
G.add_edges_from([('A','O','d'),('A','O','c'),('I','A','i')])
Lrg = FiniteInverseCategory(G,[(('I','O','id'),('I','O','ic'))],ficname='Lrg')

In [ ]:
Lrg.ficname

To view the set of morphisms $\mathcal{L}_{\text{rg}} (I,O)$ one does this...

In [ ]:
Lrg.Hom[('I','O')]

...and to get the domain and the codomain of a morphism:

In [ ]:
assert Lrg.cod['i'] == 'A'
assert Lrg.dom['id'] == 'I'

Let us also look at how the extend method works.

The following cell defines the fic given by two parallel arrows $A \rightrightarrows O$...

In [ ]:
H = nx.MultiDiGraph()
H.add_edges_from([('A','O','d'),('A','O','c')])
L_A = FiniteInverseCategory(H)

...and then extend by adding an arrow $i : I \to A$ and the relation $ic = id$...

In [ ]:
L_A.extend([('I','A','i')],[(('I','O','id'),('I','O','ic'))],'Lrg_2')

which gives us a fic that is isomorphic to $\mathcal{L}_{\text{rg}}$.

To interpret $\mathcal{L}_{\text{rg}}$ as a $\Sigma$-type in UniMath:

In [ ]:
Lrg.UniMathDataShape()

## FIC structures

Now let us define an $\mathcal{L}_{\text{rg}}$-structure $M$, i.e. a functor $M: \mathcal{L}_{\text{rg}} \rightarrow \textbf{Set}$

In [ ]:
obval = {'O':['x','y'], 'A':['p'],'I':['s']}
morval = {('A','O','d'):TypedFunction(obval['A'],obval['O'],{'p':'x'}),\
          ('A','O','c'):TypedFunction(obval['A'],obval['O'],{'p':'x'}),\
          ('I','A','i'):TypedFunction(obval['I'],obval['A'],{'s':'p'})}

In [ ]:
M = FICStructure(Lrg,obval,morval)

In [ ]:
M.strucname

To check the value of the functor on objects do:

In [ ]:
M.obval

And to interpret $M$ in UniMath do:

In [ ]:
M.UniMathDataPoint()

Let us also define the yoneda $\mathcal{L}_{\text{rg}}$-structure $\textbf{y}I$ and the reduced yoneda $\mathcal{L}_{\text{rg}}$-structure $\partial I$

In [ ]:
Y = M
Y = M.yoneda('I')
Yhat = M.reducedyoneda('I')

In [ ]:
Y.obval

In [ ]:
Yhat.obval

Let us now see an example of a collage, i.e. a process of producing a fic $\mathcal{L}*M$ from a fic $\mathcal{L}$ and an $\mathcal{L}$-structure $M$. 

In particular let us define $\mathcal{L}_A$...

In [ ]:
G = nx.MultiDiGraph()
G.add_edges_from([('A','O','d'),('A','O','c')])
L_A = FiniteInverseCategory(G,ficname='L_A')

and the $\mathcal{L}_A$-structure $E$...

In [ ]:
obval = {'O':['x'], 'A':['p']}
morval = {('A','O','d'):TypedFunction(obval['A'],obval['O'],{'p':'x'}),\
          ('A','O','c'):TypedFunction(obval['A'],obval['O'],{'p':'x'})}
E = FICStructure(L_A,obval,morval,strucname='E')

...and take the collage $\mathcal{L}_A * E$ of $E$.

In [ ]:
L_A_E = E.collage()

In [ ]:
L_A_E.ficname

As is to be expected, the fic $\mathcal{L}_A * E$ is isomorphic to $\mathcal{L}_{\text{rg}}$.

In [ ]:
print(L_A_E.Hom[('L_AE','O')])
print(L_A_E.Hom[('A','O')])
print(L_A_E.relations)

## FIC Transformations

Let us define a natural transformation $\alpha : \partial I \to \mathcal{L}_{\text{rg}}$

In [ ]:
YhatO = Yhat.obval['O']
MO = M.obval['O']
YhatA = Yhat.obval['A']
MA = M.obval['A']
# the functions corresponding to the data for the natural transformation
a_O = TypedFunction(YhatO,MO,{('I', 'O', 'id'):'x'})
a_A = TypedFunction(YhatA,MA,{('I', 'A', 'i'):'p'})
# definiing the FICStructureHomomorphism object
alpha = FICStructureHomomorphism(Lrg,Yhat, M,{'O':a_O,'A':a_A})

In [ ]:
alpha.funcval['O'].app

## A "Data Shape" toy example

Now for a slightly more realistic toy example

In [ ]:
G = nx.MultiDiGraph()
G.add_edges_from([('Age','Team','a'),('Gender','Team','g'),('Name','Gender','t1'),
                  ('Name','Age','t2')])
L = FiniteInverseCategory(G)

In [ ]:
namelist = ['John','Bob','Jane','Pia']
agelist = ['x_'+str(n) for n in range(1,100)]
genderlist = ['M','F','NA']
teamlist = ['Bucks','Bulls']

a = {}
for n in range(1,51):
        a['x_'+str(n)] = 'Bucks'
for n in range(51,100):
        a['x_'+str(n)] = 'Bulls'
a = TypedFunction(agelist,teamlist,a)
g = TypedFunction(genderlist,teamlist,{'M':'Bucks','F':'Bulls','NA':'Bulls'})
t1 = TypedFunction(namelist,genderlist,{'John':'M','Jane':'F','Pia':'NA','Bob':'M'})
t2 = TypedFunction(namelist,agelist,{'John':'x_26','Jane':'x_28','Pia':'x_22','Bob':'x_68'})

In [ ]:
obdata = {'Name':namelist,'Age':agelist, 'Gender':genderlist,'Team':teamlist}
mordata = {('Age', 'Team', 'a'):a,('Gender', 'Team', 'g'):g,
           ('Name', 'Gender', 't1'):t1,('Name', 'Age', 't2'):t2}
M = FICStructure(L,obdata,mordata)

In [ ]:
M.morval[('Name','Team','t2a')].app['John']

In [ ]:
C_Name = M 
C = C_Name.yoneda('Name')

In [ ]:
r_Name = TypedFunction(C.obval['Name'],M.obval['Name'],{'id_Name':'John'})
r_Age = TypedFunction(C.obval['Age'],M.obval['Age'],{('Name', 'Age', 't2'):'x_26'})
r_Gender = TypedFunction(C.obval['Gender'],M.obval['Gender'],{('Name', 'Gender', 't1'):'M'})
r_Team = TypedFunction(C.obval['Team'],M.obval['Team'],{('Name', 'Team', 't1g'):'Bucks',
                                                        ('Name', 'Team', 't2a'):'Bucks'})

In [ ]:
row = FICStructureHomomorphism(L,C,M,{'Name':r_Name,'Age':r_Age,'Gender':r_Gender,
                                      'Team':r_Team})

In [ ]:
row.funcval['Name'].app['id_Name']

The `row` is so-called because it is, as I have suggested, the replacement for the concept of a "row" in the paradigm of tabular data.

Thus, for example, the "rows" of a data shape are simply a class of certain natural transformations.

In any case let us write a "data shape" version of the toy "database" in UniMath.

In [ ]:
M.UniMathDataPoint('Teams_datashape')

## $h$-Signatures

In [ ]:
from hsig import hSignature

Let us now carry out a basic demonstration of the class of $h$-signatures, which can be thought of as fics with extra structure that makes them suitable to be interpreted as families of types of a certain $h$-level, in the sense of the Univalent Foundations of mathematics (i.e. as $n$-groupoids or homotopy $n$-types).

Let us define the same fic $\mathcal{L}_A := A \rightrightarrows O$ as above...

In [ ]:
H = nx.MultiDiGraph()
H.add_edges_from([('A','O','d'),('A','O','c')])
L_A = FiniteInverseCategory(H,ficname='L_A')

...and initialize it as an $h$-signature:

In [ ]:
hL_A = hSignature(L_A)

Notice that the name of $h\mathcal{L}_A$ is not the same as its underlying fic:

In [ ]:
assert hL_A.ficname != hL_A.hsigname

Let us now define $h\mathcal{L}_{\text{rg}}$, which is simply $\mathcal{L}_{\text{rg}}$ with specified $h$-levels for each object, added simply by adding nodes to the fic with the attribute `'hlevel'` as below...

In [ ]:
hLrg = hSignature(hL_A.extend([('I','A','i')],[(('I','O','id'),('I','O','ic'))],[('I', {'hlevel':1}),
                                                                              ('O', {'hlevel':3}),
                                                                              ('A', {'hlevel':2})],'hLrg'))

...and here is how to view the $h$-levels:

In [ ]:
hLrg.hlevel

We can now add an isomorphism sort for $O$...

In [ ]:
hLrg = hLrg.add_iso_sort('O')

which will be one less than the $h$-level of $O$...

In [ ]:
hLrg.hlevel['iso_O']

...and have the source and target maps $s_O,t_O :\:\cong_O \:\to O$, as here:

In [ ]:
hLrg.Hom[('iso_O','O')]

We can also add a reflexivity sort for $O$...

In [ ]:
hLrg = hLrg.add_refl_sort('O')

which will be two less than the $h$-level of $O$...

In [ ]:
hLrg.hlevel['refl_O']

...and have just a single map $\rho_O s_O = \rho_Ot_O$ into $O$, as here:

In [ ]:
hLrg.Hom[('refl_O','O')]

Finally, we can add a transport sort along the "domain" map $d : A \to O$...

In [ ]:
hLrg = hLrg.add_trans_sort(('A','O','d'))

...which will be one less than the $h$-level of $A$...

In [ ]:
hLrg.hlevel['trans_d']

and have three maps down to $O$...

In [ ]:
hLrg.Hom[('trans_d','O')]

...which can be thought of the three variables $p, a, b$ in the equation

$$\mathrm{trans}_p (a) = b$$

expressing that $b$ is the transport of $a$ along $p$